# Get SP500 data from Yahoo Finance and usd to euro conversion rate


In [4]:
import yfinance as yf
import pandas as pd

In [21]:


# Download SP500 data
sp500_data = yf.download('^GSPC', start='2023-01-01', end='2025-04-23', interval='1d')
# Calculate the relative daily difference for each column
sp500_data_diff = sp500_data.pct_change()
sp500_data_diff

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
2023-01-03,NaN,NaN,NaN,NaN,NaN
2023-01-04,0.007539,-0.001367,0.005651,-0.003356,0.114909
2023-01-05,-0.011646,-0.008629,-0.003499,-0.000161,-0.117948
2023-01-06,0.022841,0.017306,0.001878,-0.004263,0.007734
2023-01-09,-0.000768,0.011361,0.021226,0.022872,0.098943
...,...,...,...,...,...
2025-04-15,-0.001728,-0.001658,0.005304,-0.005507,-0.141973
2025-04-16,-0.022408,-0.015259,-0.030753,-0.014087,0.067323


In [22]:
# Download USD to Euro conversion rate data
usd_to_euro_data = yf.download('EURUSD=X', start='2023-01-01', end='2025-04-23', interval='1d')
usd_to_euro_data_diff = usd_to_euro_data.pct_change()
# Extract daily close prices
usd_to_euro_data_diff

[*********************100%***********************]  1 of 1 completed

Price,Close,High,Low,Open,Volume
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X
Date,,,,,
2023-01-02,NaN,NaN,NaN,NaN,NaN
2023-01-03,-0.002990,-0.002777,-0.012363,-0.002990,NaN
2023-01-04,-0.012255,-0.004784,0.002320,-0.012255,NaN
2023-01-05,0.005642,0.000106,-0.002881,0.005642,NaN
2023-01-06,-0.007934,-0.000977,-0.002883,-0.007934,NaN
...,...,...,...,...,...
2025-04-14,0.007577,-0.004102,0.004339,0.007577,NaN
2025-04-15,-0.000306,-0.004107,-0.001241,-0.000306,NaN


In [23]:
all_data=pd.DataFrame()
all_data['close'] = sp500_data['Close']
all_data['close_diff'] = sp500_data_diff['Close']
all_data['usdeur'] = usd_to_euro_data['Close']
all_data['usdeur_diff'] = usd_to_euro_data_diff['Close']
all_data


,close,close_diff,usdeur,usdeur_diff
Date,,,,
2023-01-03,3824.139893,NaN,1.067771,-0.002990
2023-01-04,3852.969971,0.007539,1.054685,-0.012255
2023-01-05,3808.100098,-0.011646,1.060637,0.005642
2023-01-06,3895.080078,0.022841,1.052222,-0.007934
2023-01-09,3892.090088,-0.000768,1.065632,0.012745
...,...,...,...,...
2025-04-15,5396.629883,-0.001728,1.133967,-0.000306
2025-04-16,5275.700195,-0.022408,1.129267,-0.004144
2025-04-17,5282.700195,0.001327,1.139679,0.009220


In [29]:
start_date = '2023-08-01'  # Define the start date
initial_eur = 100  # Initial value in EUR

# Filter data starting from the defined start date
all_data_filtered = all_data.loc[start_date:].copy()

# Calculate the cumulative value in EUR
all_data_filtered['cumulative_eur'] = initial_eur * (1 + all_data_filtered['close_diff']).cumprod() * (1 + all_data_filtered['usdeur_diff']).cumprod() #/ all_data_filtered['usdeur'])
all_data_filtered.ffill(inplace=True)  # Forward fill to handle any NaN values
all_data_filtered['cumulative_eur_diff'] = all_data_filtered['cumulative_eur'].pct_change(fill_method=None)

# Add the cumulative_eur column back to the original all_data DataFrame
all_data_filtered

,close,close_diff,usdeur,usdeur_diff,cumulative_eur,cumulative_eur_diff
Date,,,,,,
2023-08-01,4576.729980,-0.002665,1.099759,-0.002419,99.492187,NaN
2023-08-02,4513.390137,-0.013840,1.100788,0.000936,98.207065,-0.012917
2023-08-03,4501.890137,-0.002548,1.094415,-0.005789,97.389722,-0.008323
2023-08-04,4478.029785,-0.005300,1.095158,0.000679,96.939330,-0.004625
2023-08-07,4518.439941,0.009024,1.100267,0.004665,98.270437,0.013731
...,...,...,...,...,...,...
2025-04-15,5396.629883,-0.001728,1.133967,-0.000306,121.455921,-0.002033
2025-04-16,5275.700195,-0.022408,1.129267,-0.004144,118.242212,-0.026460
2025-04-17,5282.700195,0.001327,1.139679,0.009220,119.490741,0.010559


In [36]:
import plotly.express as px

# Create a bar chart for cumulative_eur_diff
fig = px.bar(
    all_data_filtered,
    x=all_data_filtered.index,
    y='cumulative_eur_diff',
    title='Cumulative EUR Difference by Date',
    labels={'cumulative_eur_diff': 'Cumulative EUR Difference', 'index': 'Date'},
)

# Customize the layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Cumulative EUR Difference (%)",
    yaxis_tickformat="%",  # Format y-axis as percentage
    height=600,
    width=1000,
    yaxis_zeroline=True,
    yaxis_zerolinecolor="black",
    yaxis_zerolinewidth=2,
)

fig.show()